In [ ]:
!pip install httpx
!pip install pydantic_ai

In [ ]:
import json
from tqdm import tqdm
from typing import Optional, Dict
import httpx
from typing import Optional
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from datetime import datetime, timezone, timedelta
import re

def parse_json_from_code_block(output: str):
    """
    Parse a JSON object from a Markdown code block.

    This function removes Markdown code fences (```json ... ```)
    from a string and parses the cleaned content as JSON.

    Args:
        output (str): The string containing a JSON object, possibly
                      wrapped in Markdown code block syntax.

    Returns:
        dict: Parsed JSON object.

    Raises:
        json.JSONDecodeError: If the cleaned string is not valid JSON.
    """
    # Remove markdown  ```json ... ```
    cleaned = re.sub(r"^```json|```$", "", output.strip(), flags=re.MULTILINE).strip()

    # Parsing
    return json.loads(cleaned)

class Billing:
    """
    Extract and store billing information from HTTP response headers and body.

    Attributes:
        cost (Optional[float]): Response cost parsed from headers.
        key_spend (Optional[float]): Key spend parsed from headers.
        proxy_request_id (Optional[str]): Request ID extracted from the response body.
        created (Optional[str]): Creation timestamp (converted from Unix time to ISO-8601).
        raw_headers (Dict[str, str]): Raw response headers.
    """
    def __init__(self, headers: Dict[str, str], body: Dict[str, str]):
        """
        Initialize Billing object by parsing headers and response body.

        Args:
            headers (Dict[str, str]): HTTP response headers.
            body (Dict[str, str]): HTTP response body.
        """
        self.cost: Optional[float] = self._parse_float(headers.get("x-litellm-response-cost"))
        self.key_spend: Optional[float] = self._parse_float(headers.get("x-litellm-key-spend"))
        self.proxy_request_id: Optional[str] = body.get('id')
        self.created: Optional[str] = self._parse_unix_time(body.get('created'))
        self.raw_headers: Dict[str, str] = headers

    @staticmethod
    def _parse_float(value: Optional[str]) -> Optional[float]:
        """
        Safely parse a string into a float.

        Args:
            value (Optional[str]): String to parse.

        Returns:
            Optional[float]: Parsed float value, or None if parsing fails.
        """
        try:
            return float(value) if value is not None else None
        except ValueError:
            return None

    @staticmethod
    def _parse_unix_time(value: int) -> Optional[str]:
        """
        Convert Unix timestamp to ISO-8601 formatted string in UTC-3 timezone.

        Args:
            value (int): Unix timestamp.

        Returns:
            Optional[str]: ISO-8601 formatted datetime string, or None if invalid.
        """
        try:
            dt_utc = datetime.fromtimestamp(value, tz=timezone.utc)
            return dt_utc.astimezone(timezone(timedelta(hours=-3))).isoformat()
        except ValueError:
            return None

    def to_dict(self) -> Dict[str, Optional[str]]:
        """
        Convert the billing information into a dictionary.

        Returns:
            dict: Dictionary with billing information.
        """
        return {
            "cost": self.cost,
            "key_spend": self.key_spend,
            "proxy_request_id": self.proxy_request_id,
            "created": self.created
        }


class CustomHTTPClient(httpx.AsyncClient):
    """
    Custom HTTP client that extends `httpx.AsyncClient` to capture billing information.

    Attributes:
        last_billing (Optional[Billing]): The last parsed billing information
                                          from a response, or None if not available.
    """
    def __init__(self, *args, **kwargs):
        """
        Initialize the custom HTTP client.

        Args:
            *args: Positional arguments passed to `httpx.AsyncClient`.
            **kwargs: Keyword arguments passed to `httpx.AsyncClient`.
        """
        super().__init__(*args, **kwargs)
        self.last_billing: Optional[Billing] = None

    async def send(self, request: httpx.Request, **kwargs) -> httpx.Response:
        """
        Send an HTTP request and capture billing information from the response.

        Args:
            request (httpx.Request): The HTTP request to send.
            **kwargs: Additional arguments for `httpx.AsyncClient.send`.

        Returns:
            httpx.Response: The HTTP response object.
        """
        response = await super().send(request, **kwargs)
        body = response.json()
        self.last_billing = Billing(response.headers, body)
        return response


custom_client = CustomHTTPClient()

# Model config
model = OpenAIModel(
        "<path_to_model>",
        provider=OpenAIProvider(
            api_key="<api_key>",
            base_url="<base_url>",
            http_client=custom_client
        ),
    )

# Create the AI agent
agent = Agent(model,
             system_prompt=""" Você é um especialista em anonimização de textos médicos e sua tarefa é avaliar a qualidade da anonimização de um prontuário médico baseado em três critérios que são:
            Desempenho técnico da anonimização, Perda de Informação e Risco de reidentificação.

            Entrada:
            Você receberá três versões do mesmo prontuário médico:

            Original: Prontuário médico sem anonimização.
            Anonimizado por especialista: Versão do prontuário que foi anonimizada manualmente por um especialista humano.
            Anonimizado por modelo: Versão do prontuário que foi anonimizada por um modelo de linguagem.

            Tarefa:
            Avalie a versão anonimizada pelo modelo de linguagem em relação ao prontuário original e à versão anonimizada pelo especialista, utilizando os seguintes critérios:

            Desempenho técnico da anonimização:

            Avalie se todas as informações identificáveis (nomes, datas, endereços, telefone, nome de hospital, números de identificação de médicos e pacientes, etc.) foram corretamente removidas ou mascaradas.
            Score 0 (péssimo) a 10 (perfeito).

            Perda de informação (Information Loss):

            Avalie se a anonimização preservou o máximo possível de informações úteis para análise clínica e treinamento de um modelo de deep learning em tarefas médicas,
            mantendo a legibilidade e o valor semântico do texto comparado ao Original.
            Score 0 (informação completamente perdida) a 10 (mínima perda de informação).

            Capacidade de reidentificação humana (Human Reidentification Risk):

            Avalie se um humano poderia reidentificar o paciente com base nas informações restantes no prontuário anonimizado. Quanto maior o risco, menor a nota.
            Score 0 (altíssimo risco de reidentificação) a 10 (nenhuma chance de reidentificação).
            Formato de saída:
            Forneça a avaliação no seguinte formato JSON:

            {
              "Desempenho": {
                "score": <score>,
                "text": "<explicação detalhada da nota atribuída>"
              },
              "Perda": {
                "score": <score>,
                "text": "<explicação detalhada da nota atribuída>"
              },
              "Reidentificação": {
                "score": <score>,
                "text": "<explicação detalhada da nota atribuída e entidades problemáticas>"
              }
            }
            Agora, analise os três prontuários fornecidos e gere a saída JSON.
            No campo de texto dê um diagnóstico preciso e aprofundado explicando em detalhes o porquê da nota recebida.

            """,)



In [ ]:
### Path to data generated during model inference on previous jupyter notebook
generative_test_data_path = 'bert_generative_predictions.json'

# Read data
with open(generative_test_data_path, 'r') as file:
    generative_test_set = json.load(file)

len(generative_test_set), generative_test_set[0]

In [ ]:
list_preds = []
for generative_pred in tqdm(generative_test_set[-200:]):

    input_text = "<<Prontuário não anonimizado:>> " + generative_pred['text'] + '\n\n <<Prontuário anonimizado por especialista:>> ' + generative_pred['masked_text'] + '\n\n <<Prontuário anonimizado por modelo de linguagem:>> ' + generative_pred['prediction']

    try:
        # Run model
        results = await agent.run(input_text)
        list_preds.append(parse_json_from_code_block(results.output))
    except:
        print('JSON format error!')

len(list_preds), list_preds[0]

In [ ]:
### Results
list_performance = [pred['Desempenho']['score'] for pred in list_preds]
list_loss = [pred['Perda']['score'] for pred in list_preds]
list_reidentification = [pred['Reidentificação']['score'] for pred in list_preds]

# Calculate the average of each list
mean_performance = sum(list_performance) / len(list_performance) if list_performance else 0
mean_loss = sum(list_loss) / len(list_loss) if list_loss else 0
mean_reidentification = sum(list_reidentification) / len(list_reidentification) if list_reidentification else 0

# Print Results
print(list_performance)
print(list_loss)
print(list_reidentification)
print('')
print(f"Average of Performance: {mean_performance:.2f}")
print(f"Average of Information loss: {mean_loss:.2f}")
print(f"Average of Reidentification Risk: {mean_reidentification:.2f}")

In [ ]:
# Number of reviews used as input to the model
n_reviews = 25

# Prepare data to generate the reports
list_performance_text = [pred['Desempenho']['text'] for pred in list_preds[:n_reviews]]
list_loss_text = [pred['Perda']['text'] for pred in list_preds[:n_reviews]]
list_reidentification_text = [pred['Reidentificação']['text'] for pred in list_preds[:n_reviews]]

list_performance_text_final = 'Desempenho técnico da anonimização \n\n<AVALIAÇAO> \n' + '\n\n<AVALIAÇAO> \n'.join(list_performance_text)
list_loss_text_final = 'Perda de informação \n\n<AVALIAÇAO> \n' + '\n\n<AVALIAÇAO> \n'.join(list_loss_text)
list_reidentification_text_final = 'Risco de reidentificação humana \n\n<AVALIAÇAO> \n' + '\n\n<AVALIAÇAO> \n'.join(list_reidentification_text)
print(list_performance_text_final)

In [ ]:
# Model that generates the reports
agent_sum = Agent(model,
             system_prompt=""" Você é um especialista em privacidade de dados médicos e anonimização de textos clínicos.
             Sua tarefa é analisar uma lista de avaliações sobre prontuários médicos que foram anonimizados por um modelo de linguagem.

             Cada entrada da lista representa a avaliação de um prontuário que leva em consideração um desses três aspectos:

             1. **Desempenho técnico da anonimização** – quão bem as informações sensíveis foram removidas ou mascaradas.
             2. **Perda de informação** – o quanto a utilidade clínica do texto foi preservada após a anonimização.
             3. **Risco de reidentificação humana** – a possibilidade de que um humano consiga identificar o paciente com base nas informações restantes.

             Sua tarefa é gerar um relatório que:

             - Identifique **tendências gerais** nas avaliações para o critério especificado.
             - Destaque **pontos fortes e fracos recorrentes** destacados nas avaliações para o critério especificado.
             - Indique os principais problemas destacados nas avaliações para o critério especificado.

             No início do texto estará indicado qual critério deverá ser avaliado.

             Formato da saída:
             - Um parágrafo ou dois com até 300 palavras.
             - Não repita todos os diagnósticos, mas **sintetize os achados globais**.

             Diagnósticos de entrada:
             <<INSIRA AQUI A LISTA DE AVALIAÇÕES GERADAS>>
             """,)

In [ ]:
# Generate report for each criteria
list_reports = []
for input_str in [list_performance_text_final, list_loss_text_final, list_reidentification_text_final]:

    # Run model
    results_ = await agent_sum.run(input_str)
    list_reports.append(results_.output)

len(list_reports), list_reports[-1]

In [ ]:
# Output path
caminho_arquivo = "report_bert.json"

# Save JSON
with open(caminho_arquivo, "w", encoding="utf-8") as f:
    json.dump(list_reports, f, indent=2, ensure_ascii=False)